In [119]:
import pandas as pd
import numpy as np
import os

In [120]:
!pip install tqdm
from tqdm import tqdm

You should consider upgrading via the '/Users/jennifer.l/env/bin/python3 -m pip install --upgrade pip' command.


In [121]:
import warnings
warnings.filterwarnings('ignore')

In [122]:
def convert_to_log(df, col_name):
    """Convert column to log space.

    Defining log as log(x + EPSILON) to avoid division by zero.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.
    col_name : str
        Name of column in df to convert to log.

    Returns
    -------
    np.ndarray
        Values of column in log space

    """
    # This is to avoid division by zero while doing np.log10
    EPSILON = 1
    return np.log10(df[col_name].values + EPSILON)


def convert_to_percentile(df, col_name):
    """Convert column to percentile.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.
    col_name : str
        Name of column in df to convert to percentile.

    Returns
    -------
    pd.Series
        Column converted to percentile from 1 to 100

    """
    return pd.qcut(df[col_name].rank(method='first'), 100,
                   labels=range(1, 101))


In [123]:
"""
Functions for creating features.
"""


def get_dem_features(df):
    """Get demographic features.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.

    Returns
    -------
    list
        List of demographic features.

    """
    dem_features = []
    prefix = 'dem_'
    for col in df.columns:
        if prefix == col[:len(prefix)]:
            if 'race' not in col:
                dem_features.append(col)
    return dem_features


def get_comorbidity_features(df):
    """Get comorbidity features.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.

    Returns
    -------
    list
        List of comorbidity features.

    """
    comorbidity_features = []
    comorbidity_sum = 'gagne_sum_tm1'
    suffix_elixhauser = '_elixhauser_tm1'
    suffix_romano = '_romano_tm1'

    for col in df.columns:
        if col == comorbidity_sum:
            comorbidity_features.append(col)
        elif suffix_elixhauser == col[-len(suffix_elixhauser):]:
            comorbidity_features.append(col)
        elif suffix_romano == col[-len(suffix_romano):]:
            comorbidity_features.append(col)
        else:
            continue
    return comorbidity_features


def get_cost_features(df):
    """Get cost features.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.

    Returns
    -------
    list
        List of cost features.

    """
    cost_features = []
    prefix = 'cost_'
    for col in df.columns:
        if prefix == col[:len(prefix)]:
            # 'cost_t', 'cost_avoidable_t' are outcomes, not a features
            if col not in ['cost_t', 'cost_avoidable_t']:
                cost_features.append(col)
    return cost_features


def get_lab_features(df):
    """Get lab features.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.

    Returns
    -------
    list
        List of lab features.

    """
    lab_features = []
    suffix_labs_counts = '_tests_tm1'
    suffix_labs_low = '-low_tm1'
    suffix_labs_high = '-high_tm1'
    suffix_labs_normal = '-normal_tm1'
    for col in df.columns:
        # get lab features
        if suffix_labs_counts == col[-len(suffix_labs_counts):]:
            lab_features.append(col)
        elif suffix_labs_low == col[-len(suffix_labs_low):]:
            lab_features.append(col)
        elif suffix_labs_high == col[-len(suffix_labs_high):]:
            lab_features.append(col)
        elif suffix_labs_normal == col[-len(suffix_labs_normal):]:
            lab_features.append(col)
        else:
            continue
    return lab_features


def get_med_features(df):
    """Get med features.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.

    Returns
    -------
    list
        List of med features.

    """
    med_features = []
    prefix = 'lasix_'
    for col in df.columns:
        if prefix == col[:len(prefix)]:
            med_features.append(col)
    return med_features


def get_all_features(df, verbose=False):
    """Get all features.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.
    verbose : bool
        Print statistics of features.

    Returns
    -------
    list
        List of all features.

    """
    dem_features = get_dem_features(df)
    comorbidity_features = get_comorbidity_features(df)
    cost_features = get_cost_features(df)
    lab_features = get_lab_features(df)
    med_features = get_med_features(df)

    x_column_names = dem_features + comorbidity_features + cost_features + \
                     lab_features + med_features

    if verbose:
        print('Features breakdown:')
        print('   {}: {}'.format('demographic', len(dem_features)))
        print('   {}: {}'.format('comorbidity', len(comorbidity_features)))
        print('   {}: {}'.format('cost', len(cost_features)))
        print('   {}: {}'.format('lab', len(lab_features)))
        print('   {}: {}'.format('med', len(med_features)))
        print(' {}: {}'.format('TOTAL', len(x_column_names)))

    return x_column_names

In [124]:
def split_by_id(df, id_field='ptid', frac_train=.6):
    """Split the df by id_field into train/holdout deterministically.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.
    id_field : str
        Split df by this column (e.g. 'ptid').
    frac_train : float
        Fraction assigned to train. (1 - frac_train) assigned to holdout.

    Returns
    -------
    pd.DataFrame
        Data dataframe with additional column 'split' indication train/holdout

    """
    ptid = np.sort(df[id_field].unique())
    print("Splitting {:,} unique {}".format(len(ptid), id_field))

    # deterministic split
    rs = np.random.RandomState(0)
    perm_idx = rs.permutation(len(ptid))
    num_train = int(frac_train*len(ptid))

    # obtain train/holdout
    train_idx = perm_idx[:num_train]
    holdout_idx  = perm_idx[num_train:]
    ptid_train = ptid[train_idx]
    ptid_holdout  = ptid[holdout_idx]
    print(" ...splitting by patient: {:,} train, {:,} holdout ".format(
      len(ptid_train), len(holdout_idx)))

    # make dictionaries
    train_dict = {p: "train" for p in ptid_train}
    holdout_dict  = {p: "holdout"  for p in ptid_holdout}
    split_dict = {**train_dict, **holdout_dict}

    # add train/holdout split to each
    split = []
    for e in df[id_field]:
        split.append(split_dict[e])
    df['split'] = split

    return df

In [125]:
def load_data_df():
    """Load data dataframe.

    Returns
    -------
    pd.DataFrame
        DataFrame to use for analysis.
    """
    
    data_df = pd.read_csv("/Users/jennifer.l/Downloads/dissecting-bias-master-data/data/data_new.csv")
    data_df = data_df.reset_index();
    return data_df

In [256]:
def get_Y_x_df(df, verbose):
    """Get dataframe with relevant x and Y columns.

    Parameters
    ----------
    df : pd.DataFrame
        Data dataframe.
    verbose : bool
        Print statistics of features.

    Returns
    -------
    all_Y_x_df : pd.DataFrame
        Dataframe with x (features) and y (labels) columns
    x_column_names : list
        List of all x column names (features).
    Y_predictors : list
        All labels (Y) to predict.

    """
    # cohort columns
    cohort_cols = ['index']

    # features (x)
    x_column_names = get_all_features(df, verbose)

    # include log columns
    df['log_cost_t'] = convert_to_log(df, 'cost_t')
    df['log_cost_avoidable_t'] = convert_to_log(df, 'cost_avoidable_t')

    # labels (Y) to predict
    Y_predictors = ['gagne_sum_t']

    # redefine 'race' variable as indicator
    df['dem_race_black'] = np.where(df['race'] == 'black', 1, 0)
    df['dem_race_white'] = np.where(df['race'] == 'white', 1, 0)

    # additional metrics used for table 2 and table 3
    table_metrics = ['dem_race_black', 'dem_race_white', 'risk_score_t', 'program_enrolled_t',
                     'cost_t', 'cost_avoidable_t']

    # combine all features together -- this forms the Y_x df
    all_Y_x_df = df[cohort_cols + x_column_names + Y_predictors + table_metrics].copy()

    return all_Y_x_df, x_column_names, Y_predictors


In [257]:
def binarize_columns(df, column_name, quantile):
    df['temp'] = ""
    #df.loc[df[column_name] < df[column_name].quantile(quantile), 'temp'] = int(0)
    #df.loc[df[column_name] >= df[column_name].quantile(quantile), 'temp'] = int(1)
    
    df.loc[df[column_name] < 7, 'temp'] = int(0)
    df.loc[df[column_name] >= 7, 'temp'] = int(1)
    
    df[column_name] = df['temp']
    df[column_name] = df[column_name].astype('int')

In [247]:
# Helper Functions
def debias_weights(original_labels, protected_attributes, multipliers):
  exponents = np.zeros(len(original_labels))
  for i, m in enumerate(multipliers):
    exponents -= m * protected_attributes[i]
  weights = np.exp(exponents)/ (np.exp(exponents) + np.exp(-exponents))
  weights = np.where(original_labels > 0, 1 - weights, weights)
  return weights

In [248]:
# Helper Functions
def get_error_and_violations(y_pred, y, protected_attributes):
  acc = np.mean(y_pred != y)
  violations = []
  for p in protected_attributes:
    protected_idxs = np.where(p > 0)
    violations.append(np.mean(y_pred) - np.mean(y_pred[protected_idxs]))
  pairwise_violations = []
# QUESTION: THE FUNCTION OF PAIRWISE VIOLATIONS?
  #for i in tqdm(range(len(protected_attributes))):
    #for j in range(i+1, len(protected_attributes)):
      #protected_idxs = np.where(np.logical_and(protected_attributes[i] > 0, protected_attributes[j] > 0))
      #if len(protected_idxs[0]) == 0:
        #continue
      #pairwise_violations.append(np.mean(y_pred) - np.mean(y_pred[protected_idxs]))
  return acc, violations, pairwise_violations

In [249]:
def get_performance(y_actual, y_hat):
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    OT = sum(y_actual == 1)
    OF = sum(y_actual == 0)

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1
        
    TPR = TP / OT
    FPR = FP / OF
    TNR = TN / OF
    FNR = FN / OT
    
    
    return TPR, FPR, TNR, FNR

In [250]:
#Logistic Regression on original dataset
def original_logisticRegression(train_df, holdout_df, x_column_names, Y_predictor, protected_groups):
    
    from sklearn.linear_model import LogisticRegression
    
    X_train = np.array(train_df[x_column_names])
    y_train = np.array(train_df[Y_predictor])
    X_test = np.array(holdout_df[x_column_names])
    y_test = np.array(holdout_df[Y_predictor])
    protected_train = [np.array(train_df[g]) for g in protected_groups]
    protected_test = [np.array(holdout_df[g]) for g in protected_groups]
    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    acc, violations, pairwise_violations = get_error_and_violations(y_pred_train, y_train, protected_train)
    print("Train Accuracy", acc)
    print("Train Violation", max(np.abs(violations)), " \t\t All violations", violations)
    if len(pairwise_violations) > 0:
        print("Train Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)

    acc, violations, pairwise_violations = get_error_and_violations(y_pred_test, y_test, protected_test)
    print("Test Accuracy", acc)
    print("Test Violation", max(np.abs(violations)), " \t\t All violations", violations)
    if len(pairwise_violations) > 0:
        print("Test Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)
    
    TP, FP, TN, FN = get_performance(y_test, y_pred_test)
            
    print("true positive rate: ", TP)
    print("false positive rate: ", FP)
    print("true negative rate: ", TN)
    print("false negative rate: ", FN)
    
    print()
    print()

In [258]:
def debaised_classifier_training(train_df, holdout_df, x_column_names, Y_predictor, protected_groups):
    
    from sklearn.linear_model import LogisticRegression
    
    X_train = np.array(train_df[x_column_names])
    y_train = np.array(train_df[Y_predictor])
    X_test = np.array(holdout_df[x_column_names])
    y_test = np.array(holdout_df[Y_predictor])
    protected_train = [np.array(train_df[g]) for g in protected_groups]
    protected_test = [np.array(holdout_df[g]) for g in protected_groups]
    
    multipliers = np.zeros(len(protected_train))
    learning_rate = 1.
    n_iters = 100
    
    test_errs = [0] * 400
    test_fair_vio = [0] * 400
    
    # algorithm 1 in the paper
    
    for it in tqdm(range(n_iters)):
        weights = debias_weights(y_train, protected_train, multipliers)
        model = LogisticRegression()
        model.fit(X_train, y_train, weights)
        y_pred_train = model.predict(X_train)
        acc, violations, pairwise_violations = get_error_and_violations(y_pred_train, y_train, protected_train)
        multipliers += learning_rate * np.array(violations)

        if (it + 1) % n_iters == 0:
            print(multipliers)
            # c = -1
            #for i in tqdm(range(400)):
                #print("c = ", c)
                #weights = debias_weights(y_train, protected_train, c*multipliers)
                #model = LogisticRegression()
                #model.fit(X_train, y_train, weights)
                
            y_pred_test = model.predict(X_test)
                
            acc, violations, pairwise_violations = get_error_and_violations(y_pred_train, y_train, protected_train)
            print("Train Accuracy", acc)
            print("Train Violation", max(np.abs(violations)), " \t\t All violations", violations)
            if len(pairwise_violations) > 0:
                print("Train Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)

            acc, violations, pairwise_violations = get_error_and_violations(y_pred_test, y_test, protected_test)
            
            TP, FP, TN, FN = get_performance(y_test, y_pred_test)
            
            print("Test Accuracy", acc)
            print("true positive rate: ", TP)
            print("false positive rate: ", FP)
            print("true negative rate: ", TN)
            print("false negative rate: ", FN)
            
            print("Test Violation", max(np.abs(violations)), " \t\t All violations", violations)
            if len(pairwise_violations) > 0:
                print("Test Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)

                #test_errs[i] = acc
                #test_fair_vio[i] = max(np.abs(violations))

                print()
                print()
        
                #c += 0.01
    
    return test_errs, test_fair_vio
    
    

In [260]:
def main():
    # load data
    data_df = load_data_df()
    
    print(data_df['cost_avoidable_t'][:100])

    # subset to relevant columns
    all_Y_x_df, x_column_names, Y_predictors = get_Y_x_df(data_df, verbose=True)
    
    print(Y_predictors)
    
    # binarize labels at 55 quantile
    for column in Y_predictors:
        binarize_columns(all_Y_x_df, column, 0.97)
        
    print(sum(all_Y_x_df['gagne_sum_t'] == 1))

    # assign to 2/3 train, 1/3 holdout
    all_Y_x_df = split_by_id(all_Y_x_df, id_field='index',
                                   frac_train=.67)

    # define train, holdout
    # reset_index for pd.concat() along column
    train_df = all_Y_x_df[all_Y_x_df['split'] == 'train'].reset_index(drop=True)
    holdout_df = all_Y_x_df[all_Y_x_df['split'] == 'holdout'].reset_index(drop=True)
    
    # dealing with missing data
    for column in x_column_names:
        train_mean = train_df[column].mean()
        train_df[column].fillna(train_mean, inplace=True)
        holdout_df[column].fillna(train_mean, inplace=True)
    
    protected_groups = ['dem_race_black', 'dem_race_white']
    
    arr_err = []
    arr_vio = []
    
    for column in Y_predictors:
        print("Processing original logistic model with ", column)
        original_logisticRegression(train_df, holdout_df, x_column_names, column, protected_groups)
        
        print("Processing weighted logistic model with ", column)
        err, vio = debaised_classifier_training(train_df, holdout_df, x_column_names, column, protected_groups)
        
        arr_err.append(err)
        arr_vio.append(vio)
        
    #pd.DataFrame(arr_err).to_csv('err_rate_55.csv')
    #pd.DataFrame(arr_vio).to_csv('fair_vio_55.csv')


In [261]:
if __name__ == '__main__':
    main()


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
95       0.0
96    2800.0
97       0.0
98       0.0
99       0.0
Name: cost_avoidable_t, Length: 100, dtype: float64
Features breakdown:
   demographic: 8
   comorbidity: 34
   cost: 13
   lab: 90
   med: 4
 TOTAL: 149
['gagne_sum_t']
1404
Splitting 48,784 unique index
 ...splitting by patient: 32,685 train, 16,099 holdout 
Processing original logistic model with  gagne_sum_t
Train Accuracy 0.0329814899801132
Train Violation 0.0162542907514967  		 All violations [-0.0162542907514967, 0.0021110125260430414]
Test Accuracy 0.03329399341574011
Test Violation 0.01622307860523967  		 All violations [-0.01622307860523967, 0.002074199135110158]
true positive rate:  0.16808510638297872
false positive rate:  0.009277624928018428
true negative rate:  0.9907223750719816
false negative rate:  0.8319148936170213


Processing weighted logistic model with  gagne_sum_t


100%|█████████████████████████████████████████████████████████████| 100/100 [01:40<00:00,  1.01s/it]

[-1.61083014  0.20920523]
Train Accuracy 0.03218601805109377
Train Violation 0.01758210231320178  		 All violations [-0.01758210231320178, 0.0022834609510059157]
Test Accuracy 0.03149263929436611
true positive rate:  0.1829787234042553
false positive rate:  0.007869985283767356
true negative rate:  0.9921300147162326
false negative rate:  0.8170212765957446
Test Violation 0.016058923084646626  		 All violations [-0.016058923084646626, 0.002053211057130454]
